**Import the laibrary**

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport

c:\Users\Nadia Hussein\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
pip install ydata_profiling

  Using cached ydata_profiling-4.16.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached scipy-1.15.3-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached matplotlib-3.10.0-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
  Using cached visions-0.8.1-py3-none-any.whl.metadata (11 kB)
  Using cached numpy-2.1.3-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached htmlmin-0.1.12-py3-none-any.whl
  Using cached phik-0.12.5-cp311-cp311-win_amd64.whl.metadata (5.6 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached multimethod-1.12-py3-none-any.whl.metadata (9.6 kB)
  Using cached statsmodels-0.14.5-cp311-cp311-win_amd64.whl.metadata (9.8 kB)
  Using cached typeguard-4.4.4-py3-none-any.whl.metadata (3.3 kB)
  Using cached ImageHash-4.3.1-py2.py3-none-any.whl.metadata (8.0 kB)
  Using cached wordcloud-1.9.4-cp311-cp311-win_amd64.whl.metadata (3.5 kB)
  Using cached dacite-1.9.2-py3-none-any.whl


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
df = pd.read_excel('Employee Sample Data - A.xlsx')
print("Features Name:", df.columns.tolist())

# print first 5 rows
df.head()

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

**Exploratory Data Analysis Report**

In [3]:
profile = ProfileReport(
    df,
    title="Exploratory Data Analysis Report",
    explorative=True
)

output_filename = "employee_data_profile.html"
profile.to_file(output_filename)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 14/14 [00:00<00:00, 34.34it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
df.info()
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   EEID           1000 non-null   object        
 1   Full Name      998 non-null    object        
 2   Job Title      999 non-null    object        
 3   Department     998 non-null    object        
 4   Business Unit  1000 non-null   object        
 5   Gender         999 non-null    object        
 6   Ethnicity      993 non-null    object        
 7   Age            994 non-null    float64       
 8   Hire Date      993 non-null    datetime64[ns]
 9   Annual Salary  989 non-null    float64       
 10  Bonus %        992 non-null    float64       
 11  Country        998 non-null    object        
 12  City           998 non-null    object        
 13  Exit Date      85 non-null     datetime64[ns]
dtypes: datetime64[ns](2), float64(3), object(9)
memory usage: 109.5+ KB


,0
EEID,0
Full Name,2
Job Title,1
Department,2
Business Unit,0
Gender,1
Ethnicity,7
Age,6
Hire Date,7
Annual Salary,11


# Cleaning Dataset

In [ ]:
# Clean 'Annual Salary' column: remove '$', ',' , and convert to numeric fl
df['Annual Salary'] = df['Annual Salary'].replace({'\$': '', ',': ''}, regex=True).astype(float)


In [ ]:
df['Annual Salary'].head()

In [ ]:
# Clean 'Bonus %' column, remove % , convert to float
df['Bonus %'] = df['Bonus %'].replace({'%': ''}, regex=True).astype(float)

# Fill any missing bonus values with 0
df['Bonus %'].fillna(0, inplace=True)

df['Bonus %'].head()


In [ ]:
# convert date columns
df['Hire Date'] = pd.to_datetime(df['Hire Date'], errors='coerce').dt.date
df['Exit Date'] = pd.to_datetime(df['Exit Date'], errors='coerce').dt.date

df.head()

In [ ]:
# Modify the First 5 Rows
for i in range(5):
    df.loc[i, 'Full Name'] = f"Person {i+1}"
    df.loc[i, 'Age'] = np.random.randint(25, 55)
    df.loc[i, 'Annual Salary'] = np.random.randint(60000, 180000)
    df.loc[i, 'Department'] = 'Research & Development'
    df.loc[i, 'Job Title'] = 'Lead Scientist'
df.head()

In [ ]:
# find and print the row with the largest salary
max_salary_row = df[df['Annual Salary'] == df['Annual Salary'].max()]
print("Row with the largest salary:")
max_salary_row

In [ ]:
# group by department (Average age and salary)
department_grouped  = df.groupby('Department').agg({'Age': 'mean', 'Annual Salary': 'mean'}).rename(columns={'Age': 'Average Age', 'Annual Salary': 'Average Annual Salary'}).round(2)

department_grouped

In [ ]:
# group by Department and Ethncity
dept_ethnicity_grouped = df.groupby(['Department', 'Ethnicity']).agg({
    'Age': ['max', 'min'],
    'Annual Salary': 'median'
}).round(2)

dept_ethnicity_grouped

In [ ]:
# save the analysis to an excel file
try:
    with pd.ExcelWriter('employee_analysis.xlsx') as writer:
        df.to_excel(writer, sheet_name='Cleaned_Data', index=False)
        department_grouped.to_excel(writer, sheet_name='Department_Analysis')
        dept_ethnicity_grouped.to_excel(writer, sheet_name='Dept_Ethnicity_Analysis')
except Exception as e:
    print(f"Error saving file: {e}")

# Visualization

In [ ]:
# Set plot style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 7)

# Bar Chart, Average Salary by Department
plt.figure()
avg_salary_sorted = department_grouped['Average Annual Salary'].sort_values(ascending=False)
sns.barplot(x=avg_salary_sorted.values, y=avg_salary_sorted.index, palette="viridis")
plt.title('Average Annual Salary by Department', fontsize=16)
plt.xlabel('Average Annual Salary ($)', fontsize=12)
plt.ylabel('Department', fontsize=12)
plt.xticks(rotation=0)
plt.show()

In [ ]:
# Histogram, Distribution of Employee Age
plt.figure()
sns.histplot(df['Age'], bins=20, kde=True, color='skyblue')
plt.title('Distribution of Employee Age', fontsize=16)
plt.xlabel('Age', fontsize=12)
plt.ylabel('Number of Employees', fontsize=12)
plt.tight_layout()
plt.show()


In [ ]:

# Pie Chart, Employee Distribution by Country
plt.figure()
country_counts = df['Country'].value_counts()
# Group smaller countries into 'Other' to keep the pie chart clean
threshold = 20
main_countries = country_counts[country_counts >= threshold]
other_count = country_counts[country_counts < threshold].sum()
if other_count > 0:
    main_countries['Other'] = other_count

plt.pie(main_countries, labels=main_countries.index, autopct='%1.1f%%', startangle=140, colors=sns.color_palette("Set2"))
plt.title('Employee Distribution by Country', fontsize=16)
plt.ylabel('')
plt.axis('equal')
plt.show()